In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "name": "SPY.ipynb",
      "provenance": [],
      "collapsed_sections": []
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "1bC0z4LPD2ez"
      },
      "source": [
        ""
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "sk3haDSwYT63"
      },
      "source": [
        "import pandas as pd\n",
        "import numpy as np\n",
        "\n",
        "import matplotlib.pyplot as plt\n",
        "%matplotlib inline\n",
        "\n",
        "from matplotlib.pylab import rcParams\n",
        "rcParams['figure.figsize']=20,10\n",
        "from keras.models import Sequential\n",
        "from keras.layers import LSTM,Dropout,Dense\n",
        "\n",
        "\n",
        "from sklearn.preprocessing import MinMaxScaler"
      ],
      "execution_count": 1,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "JWS1KzhDD6D5"
      },
      "source": [
        "Importing pandas, numpy, and matplotlib packages"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "FxlQE3AEYUmT",
        "outputId": "b616f752-e24d-4632-b093-417496423549"
      },
      "source": [
        "\n",
        "df2=pd.read_csv(\"Newdata\")\n",
        "df2.head()\n",
        "print(format(df2.Date[0]))\n",
        "print(format(df2.Date[0]))\n"
      ],
      "execution_count": 3,
      "outputs": [
        {
          "output_type": "stream",
          "text": [
            "1993-01-29\n",
            "1993-01-29\n"
          ],
          "name": "stdout"
        }
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "y8wgY4yzETba"
      },
      "source": [
        "Analyzing closing prices within the dataframe collected"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 500
        },
        "id": "E4AV8TC-YhFY",
        "outputId": "097eed27-2338-42be-be81-178b05a192d0"
      },
      "source": [
        "df2[\"Date\"]=pd.to_datetime(df2.Date,format=\"%Y-%m-%d\")\n",
        "df2.index=df2['Date']\n",
        "\n",
        "plt.figure(figsize=(16,8))\n",
        "plt.plot(df2[\"Close\"],label='Close Price history')"
      ],
      "execution_count": 6,
      "outputs": [
        {
          "output_type": "execute_result",
          "data": {
            "text/plain": [
              "[<matplotlib.lines.Line2D at 0x7f0e26615390>]"
            ]
          },
          "metadata": {
            "tags": []
          },
          "execution_count": 6
        },
        {
          "output_type": "display_data",
          "data": {
            "image/png": "...\n",
            "text/plain": [
              "<Figure size 1152x576 with 1 Axes>"
            ]
          },
          "metadata": {
            "tags": [],
            "needs_background": "light"
          }
        }
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "zxNVEZ9WYlDF"
      },
      "source": [
        "data=df2.sort_index(ascending=True,axis=0)\n",
        "new_dataset=pd.DataFrame(index=range(0,len(df2)),columns=['Date','Close'])\n",
        "\n",
        "for i in range(0,len(data)):\n",
        "    new_dataset[\"Date\"][i]=data['Date'][i]\n",
        "    new_dataset[\"Close\"][i]=data[\"Close\"][i]"
      ],
      "execution_count": 7,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "G0ZpsTosEaTS"
      },
      "source": [
        "Sorting the dataset using date time and the following filters \"Date\" and \"Close\" columns"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "bAeDm9q8Yn5N"
      },
      "source": [
        "scaler=MinMaxScaler(feature_range=(0,1))\n",
        "new_dataset.index=new_dataset.Date\n",
        "new_dataset.drop('Date',axis=1,inplace=True)\n",
        "final_dataset=new_dataset.values\n",
        "\n",
        "train_data=final_dataset[0:5670,:]\n",
        "valid_data=final_dataset[5670:,:]\n",
        "\n",
        "scaled_data=scaler.fit_transform(final_dataset)\n",
        "\n",
        "x_train_data,y_train_data=[],[]\n",
        "\n",
        "for i in range(60,len(train_data)):\n",
        "    x_train_data.append(scaled_data[i-60:i,0])\n",
        "    y_train_data.append(scaled_data[i,0])\n",
        "    \n",
        "x_train_data,y_train_data=np.array(x_train_data),np.array(y_train_data)\n",
        "\n",
        "x_train_data=np.reshape(x_train_data,(x_train_data.shape[0],x_train_data.shape[1],1))"
      ],
      "execution_count": 8,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "JVO_9yF5Ejih"
      },
      "source": [
        "Normalizing the new filtered dataset that has been created"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "Xc3bI2bWYraD"
      },
      "source": [
        "lstm_model=Sequential()\n",
        "lstm_model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train_data.shape[1],1)))\n",
        "lstm_model.add(LSTM(units=50))\n",
        "lstm_model.add(Dense(1))\n",
        "\n",
        "inputs_data=new_dataset[len(new_dataset)-len(valid_data)-60:].values\n",
        "inputs_data=inputs_data.reshape(-1,1)\n",
        "inputs_data=scaler.transform(inputs_data)\n",
        "\n",
        "lstm_model.compile(loss='mean_squared_error',optimizer='adam')\n",
        "lstm_model.fit(x_train_data,y_train_data,epochs=1,batch_size=1,verbose=2)"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "LG077ahdEnik"
      },
      "source": [
        "Creating a build and train Long short-term memory (LSTM) model"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "8EImOc8eg3XP"
      },
      "source": [
        "X_test=[]\n",
        "for i in range(60,inputs_data.shape[0]):\n",
        "    X_test.append(inputs_data[i-60:i,0])\n",
        "X_test=np.array(X_test)\n",
        "\n",
        "X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))\n",
        "predicted_closing_price=lstm_model.predict(X_test)\n",
        "predicted_closing_price=scaler.inverse_transform(predicted_closing_price)"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "wdAAc3j9EusP"
      },
      "source": [
        "Using a sample of the dataset to make a series of stock price predictions using the Long short-term memory (LSTM) model"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "TFy6Q36fhJQg"
      },
      "source": [
        "train_data=new_dataset[:5670]\n",
        "valid_data=new_dataset[5670:]\n",
        "valid_data['Predictions']=predicted_closing_price\n",
        "plt.plot(train_data[\"Close\"])\n",
        "plt.plot(valid_data[['Close',\"Predictions\"]])"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "oPSxZJDyE5k5"
      },
      "source": [
        "Visualization of the predicted stock costs with actual stock costs"
      ]
    },
    {
      "cell_type": "code",
      "metadata": {
        "id": "811FUuFahMBf"
      },
      "source": [
        "lstm_model.save(\"saved_model.h5\")"
      ],
      "execution_count": null,
      "outputs": []
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "KR8vkhJFFAHO"
      },
      "source": [
        "Creating a save file containing the Long short-term memory (LSTM) model"
      ]
    },
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "t0Qf_JbkFG9C"
      },
      "source": [
        ""
      ]
    }
  ]
}